In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [2]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
le_gender = LabelEncoder()
data['Gender'] = le_gender.fit_transform(data['Gender'])

In [5]:
ohe_geography = OneHotEncoder(handle_unknown='ignore')
geo_encoded = ohe_geography.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=ohe_geography.get_feature_names_out(['Geography']))

In [6]:
data.drop(['Geography'],axis=1,inplace=True)
data = pd.concat([data,geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [28]:
X = data.drop('EstimatedSalary',axis=1)
y = data['EstimatedSalary']

In [9]:
## Save the encoders and sscaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(le_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(ohe_geography,file)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

print(X_train[:5], y_train[:5])
print("---------------------------------")
print(X_test[:5], y_test[:5])

[[ 0.35649971  0.91324755 -0.6557859   0.34567966 -1.21847056  0.80843615
   0.64920267  0.97481699 -0.50857963  1.00150113 -0.57946723 -0.57638802]
 [-0.20389777  0.91324755  0.29493847 -0.3483691   0.69683765  0.80843615
   0.64920267  0.97481699 -0.50857963 -0.99850112  1.72572313 -0.57638802]
 [-0.96147213  0.91324755 -1.41636539 -0.69539349  0.61862909 -0.91668767
   0.64920267 -1.02583358  1.96626042 -0.99850112 -0.57946723  1.73494238]
 [-0.94071667 -1.09499335 -1.13114808  1.38675281  0.95321202 -0.91668767
   0.64920267 -1.02583358  1.96626042  1.00150113 -0.57946723 -0.57638802]
 [-1.39733684  0.91324755  1.62595257  1.38675281  1.05744869 -0.91668767
  -1.54035103 -1.02583358  1.96626042  1.00150113 -0.57946723 -0.57638802]] 9254    179093.26
1561    195978.86
1670     85891.02
6087    153080.40
6669     39488.04
Name: EstimatedSalary, dtype: float64
---------------------------------
[[-0.57749609  0.91324755 -0.6557859  -0.69539349  0.32993735  0.80843615
  -1.54035103 -1.0

In [30]:
## ANN Regression Problem

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.callbacks import EarlyStopping, TensorBoard

In [42]:
input_shape = (X_train.shape[1],)

model = Sequential([
    Input(shape=input_shape),
    Dense(64,activation='relu'),
    Dense(32,activation='relu'),
    Dense(1) #Output layer for regression with default activation as linear
])

model.compile(optimizer='adam',loss='mse',metrics=['mse'])

In [43]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [44]:
import datetime

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [45]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [46]:
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stop,tensorboard_callback])

Epoch 1/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 13308822528.0000 - mse: 13308822528.0000 - val_loss: 13698983936.0000 - val_mse: 13698983936.0000
Epoch 2/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13244000256.0000 - mse: 13244000256.0000 - val_loss: 13564605440.0000 - val_mse: 13564605440.0000
Epoch 3/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13005140992.0000 - mse: 13005140992.0000 - val_loss: 13200073728.0000 - val_mse: 13200073728.0000
Epoch 4/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 12498809856.0000 - mse: 12498809856.0000 - val_loss: 12533806080.0000 - val_mse: 12533806080.0000
Epoch 5/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 11685196800.0000 - mse: 11685196800.0000 - val_loss: 11558809600.0000 - val_mse: 11558809600.0000
Epoch 6/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 10587359232.0000 - mse: 10587359232.0000 - val_loss: 10322552832.0000 - val_mse: 10322552832.0000
Epoch 7/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 1m

In [18]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [47]:
%reload_ext tensorboard

In [48]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6007 (pid 25252), started 0:10:04 ago. (Use '!kill 25252' to kill it.)

In [49]:
test_loss, test_mse = model.evaluate(X_test, y_test)
print('Test Loss:', test_loss)
print('Test Mean Squared Error:', test_mse)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 833us/step - loss: 3370025216.0000 - mse: 3370025216.0000
Test Loss: 3370025216.0
Test Mean Squared Error: 3370025216.0
